# Generating Mordenite (MOR) structures using PORRAN

In this tutorial we will generate various MOR structures, where we substitute Si with Al using four different algorithms. The CIF for MOR was obtained from [IZA](https://europe.iza-structure.org/IZA-SC/cif/MOR.cif).

In [1]:
from porran import PORRAN

## Loading the zeolite in PORRAN

In [2]:
# Create a PORRAN object
prn = PORRAN()

# Read the input file. We will use the zeolite method to create the structure graph
# This will create a graph of the structure based on Si-O-Si bonds.
prn.init_structure('MOR.cif', graph_method='zeolite', mask_method='zeolite')


# It is also possible to load the zeolite by directly downloading the CIF from IZA
# This is done using the from_IZA_code method
prn.from_IZA_code('MOR', graph_method='zeolite', mask_method='zeolite')	

# If we wish to change the graph method, we can do so using change_graph_method
# Since radius builds a graph based on the distance matrix, we need to manually tell it which atoms to use
# In this cases we only want to substitute the Si atoms
prn.change_graph_method('radius', radius=4, mask_method=['Si'])

## Creating structures with Al substitutions

Now, we will perform substitutions using six different algorithms, and write the generated structures to different folders. This is done using the ```generate_structures``` method. Here, we need specify how many structures we want to make (```n_structures```), which algorithm for substitutions we want to use (```replace_algo```), how many atoms we want to substitute (```n_subs```) as well as which algorithm we want to use to update the CIF (```create_algo```). For certain algorithms (such as multi-clusters and chains), additional parameters need to be specified. Tt is also possible to use user-designed algorithms, which can be passed as a ```Callable``` to the method. 

In [3]:
# Random structures
random_strcts = prn.generate_structures(n_structures=10, n_subs=5, replace_algo='random', create_algo='zeolite', write=True, writepath='structures/random')

Successfully generated 10 structures in 0.099 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times


In [4]:
# Random structures obeying the Löwenstein rule
random_strcts = prn.generate_structures(n_structures=10, n_subs=5, replace_algo='random_lowenstein', create_algo='zeolite', write=True, writepath='structures/random_loewenstein')

Successfully generated 10 structures in 0.109 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times


In [5]:
# Clusters
random_strcts = prn.generate_structures(n_structures=10, n_subs=5, replace_algo='clusters', create_algo='zeolite', write=True, writepath='structures/clusters')

Successfully generated 10 structures in 0.163 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times


In [6]:
# Multiple clusters
# In this case, Al clusters are allowed to be directly next to each other (without Si in between)
# This is done by setting the make_space parameter to False
random_strcts = prn.generate_structures(n_structures=10, n_subs=5, cluster_sizes=[3,2], make_space=False, replace_algo='clusters', create_algo='zeolite', write=True, writepath='structures/multi_clusters_no_space')

# Multiple clusters with space
# In this case, Al clusters are separated by Si atoms
# This is done by setting the make_space parameter to True
random_strcts = prn.generate_structures(n_structures=10, n_subs=5, cluster_sizes=[3,2], make_space=True, replace_algo='clusters', create_algo='zeolite', write=True, writepath='structures/multi_clusters_space')

Successfully generated 10 structures in 0.111 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times
Successfully generated 10 structures in 0.098 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times


In [7]:
# Chains
# The chains algorithm might run into cases where it cannot find a suitable chain
# In this case, it will retry generating a valid structure
# For each structure, PORRAN will attempt to generate the structure 100 times by default
random_strcts = prn.generate_structures(n_structures=10, n_subs=5, chain_lengths=[2,2,1], replace_algo='chains', create_algo='zeolite', write=True, writepath='structures/chains')

Successfully generated 10 structures in 0.126 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times


In [8]:
# Maximize entropy
random_strcts = prn.generate_structures(n_structures=10, n_subs=5, replace_algo='maximize_entropy', create_algo='zeolite', write=True, writepath='structures/maximize_entropy')

Successfully generated 10 structures in 0.293 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times


## Defining custom algorithms

In PORRAN, it is possible to use your own algorithms instead of the already provided algorithms. For example, you can define your own algorithm for modification site selection (```mask_method```), graph creation (```graph_method```), modification site selection (```replace_algo```) and creating the modified structure (```create_algo```). 

Below, we will provide an example of modifying and debugging modification site selection algorithms. To illustrate this, we will develop an algorithm which selects every 10th Si atom of MOR to be replaced by an Al. When creating a custom replacement algorithm, the first argument should be the structure graph, while the second argument should be the amount of substitutions, optionally followed by other arguments. The function should return an np.array containing the indices of selected nodes.

First, we will define the new replacement algorithm.


In [9]:
import networkx as nx
import numpy as np

def replace_tenth_atom(G: nx.Graph, n_subs: int, *args, **kwargs):

    if n_subs*10 >= len(G.nodes):
        raise ValueError('Number of substitutions is too large for the structure')

    al_subs = []
    for i in range(n_subs): 
        al_subs.append(i*10+9)
    
    return np.array(al_subs)

Now, we will use the algorithm in the ```generate_structures``` method. Rather than using the string name of the function, we need to provide the actual function in the argument ```replace_algo```.

In [10]:
every10_3 = prn.generate_structures(n_structures=10, n_subs=3, replace_algo=replace_tenth_atom, create_algo='zeolite', write=True, writepath='structures/everytenth_3')

Successfully generated 10 structures in 0.097 seconds
Failed to generate 0 structures
Failed to generate new structures 0 times


Now, lets try to make a structure with 5 substitutions. Since MOR only has 48 atoms, this will fail. In order to see what causes the error, we can set the argument ```print_error``` to True.

In [11]:
every10_5 = prn.generate_structures(n_structures=10, n_subs=5, replace_algo=replace_tenth_atom, create_algo='zeolite', write=True, writepath='structures/everytenth_5', print_error=True, max_tries=1)

Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Failed to generate new structure: Number of substitutions is too large for the structure
Successfully generated 0 structures in 0.001 seconds
Failed to generate 10 structures
Failed to generate new s